In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Load required packages
import pandas as pd # work with xlsx file
import numpy as np # deals with arrays
from sklearn import preprocessing
import plotly.express as px # plotting box plots

In [4]:
## Load dataset using pandas from our drive
dataset = pd.read_excel('/content/drive/MyDrive/Data.xlsx')

# convert dataset to dataframe using pandas
df = pd.DataFrame(dataset)

# print the first 4 columns
df.head(n=4)

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,FFEEIDX,PANEL,DVDATEYR,DVDATEMM,GENDENT_M18,...,DVOF20X,DVSL20X,DVWC20X,DVOT20X,DVXP20X,DVTC20X,IMPFLAG,PERWT20F,VARSTR,VARPSU
0,2320012,102,2320012102,2320012102203301,6,-1,23,2020,2,2,...,0.0,0.0,0,0.0,202.00,285.00,3,1960.941227,2069,2
1,2320012,102,2320012102,2320012102203701,7,-1,23,2020,8,1,...,0.0,0.0,0,0.0,202.00,285.00,3,1960.941227,2069,2
2,2320012,102,2320012102,2320012102203801,7,-1,23,2020,12,2,...,0.0,0.0,0,0.0,178.20,236.92,3,1960.941227,2069,2
3,2320024,102,2320024102,2320024102207901,6,-1,23,2020,1,2,...,0.0,0.0,0,0.0,111.25,127.73,3,4054.825730,2034,3


In [5]:
# Extracting 10 -15 variables to be used 
new_df = df.filter(['DUPERSID','EVNTIDX', 'JUSTXRAY_M18','EVENTRN', 'ROOTCANL_M18','DVDATEMM','DVDATEYR','PANEL', 'DENTPROX', 'GENDENT_M18','GENDENT_M18', 'DVTC20X','PERWT20F'], axis=1)

#print the new df
new_df.head(n=5)


,DUPERSID,EVNTIDX,JUSTXRAY_M18,EVENTRN,ROOTCANL_M18,DVDATEMM,DVDATEYR,PANEL,DENTPROX,GENDENT_M18,GENDENT_M18,DVTC20X,PERWT20F
0,2320012102,2320012102203301,2,6,2,2,2020,23,2,2,2,285.00,1960.941227
1,2320012102,2320012102203701,-8,7,-8,8,2020,23,-8,1,1,285.00,1960.941227
2,2320012102,2320012102203801,2,7,2,12,2020,23,2,2,2,236.92,1960.941227
3,2320024102,2320024102207901,2,6,2,1,2020,23,2,2,2,127.73,4054.825730
4,2320024102,2320024102217101,2,7,2,10,2020,23,2,2,2,127.73,4054.825730


In [6]:
# 3.	Process missing data
## print the data format
new_df.info()

## If we carefully observe the above summary of pandas for our dataset, there are 21363 entries/observations and a total of 58 columns
## All column have 21363 entries which means there are no missing values.
## However some columns like  DENTOTHX are in object type, which means, 
## they are non-numerical. So we have to find a way to encode them to numerical values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21363 entries, 0 to 21362
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DUPERSID      21363 non-null  int64  
 1   EVNTIDX       21363 non-null  int64  
 2   JUSTXRAY_M18  21363 non-null  int64  
 3   EVENTRN       21363 non-null  int64  
 4   ROOTCANL_M18  21363 non-null  int64  
 5   DVDATEMM      21363 non-null  int64  
 6   DVDATEYR      21363 non-null  int64  
 7   PANEL         21363 non-null  int64  
 8   DENTPROX      21363 non-null  int64  
 9   GENDENT_M18   21363 non-null  int64  
 10  GENDENT_M18   21363 non-null  int64  
 11  DVTC20X       21363 non-null  float64
 12  PERWT20F      21363 non-null  float64
dtypes: float64(2), int64(11)
memory usage: 2.1 MB


In [37]:
## Check missing value in all columns
print(new_df.isnull().all())

DUPERSID        False
EVNTIDX         False
JUSTXRAY_M18    False
EVENTRN         False
ROOTCANL_M18    False
DVDATEMM        False
DVDATEYR        False
PANEL           False
DENTPROX        False
GENDENT_M18     False
GENDENT_M18     False
DVTC20X         False
PERWT20F        False
dtype: bool


In [38]:
## Check missing value in all rows
print(new_df.isnull().all(axis=1))

## We can confirm that there are no missing values

0        False
1        False
2        False
3        False
4        False
         ...  
21358    False
21359    False
21360    False
21361    False
21362    False
Length: 21363, dtype: bool


In [8]:
## Find outliers and remove them

## using pandas library describe() to find outliers

new_df.describe()[['EVENTRN', 'PERWT20F', 'DVTC20X']]

# Clearly, PERWT20F and DVTC20Xt columns have outliers. For example, the max value of PERWT20F is 92787.225297 while its mean is 12783.480960. 
# The mean is sensitive to outliers, but the fact the mean is so small compared to the max value indicates the max value is an outlier. 
# Similarly, the max value of DVTC20X is 60,000 while the mean is 453.29. 
# Since this value is entered , my best guess for the DVTC20X outlier is human error. 

,EVENTRN,PERWT20F,DVTC20X
count,21363.000000,21363.000000,21363.000000
mean,4.077002,12783.480960,453.295531
std,1.836143,8450.971341,1204.287677
min,1.000000,0.000000,0.000000
25%,3.000000,7435.709198,110.000000
50%,4.000000,11021.997937,184.380000
75%,6.000000,15799.004498,320.000000
max,7.000000,92787.225297,60000.000000


In [9]:
## we can also visualize outliers using boxplot

box_plot = px.box(new_df, y="PERWT20F", title="Find Outlier using Box plot on PERWT20F variable")

box_plot.show()

##  From the plot below, there are a lot of outliers. That thick line near 0 is the box part of our box plot.
## Above the box and upper fence are some points showing outliers. Since the chart is interactive, 
## we can zoom to get a better view of the box and points, and we can hover the mouse on the box to view of the box plot values:

In [10]:
## plot box plot for DVTC20X variable
box_plot = px.box(new_df, y="DVTC20X", title="Find Outlier using Box plot on DVTC20X variable")

box_plot.show()

## plot box plot for EVENTRN	 variable
box_plot = px.box(new_df, y="EVENTRN", title="Find Outlier using Box plot on EVENTRN variable")

box_plot.show()


In [11]:
## Lets solve the outlier problem

def impute_outliers_IQR(new_df):

   q1=new_df.quantile(0.25)

   q3=new_df.quantile(0.75)

   IQR=q3-q1

   upper = new_df[~(new_df>(q3+1.5*IQR))].max()

   lower = new_df[~(new_df<(q1-1.5*IQR))].min()

   new_df = np.where(new_df > upper,

       new_df.mean(),

       np.where(

           new_df < lower,

           new_df.mean(),

           new_df

           )

       )

   return new_df

In [12]:
## lets call the main function to transform the outliers in the two columns into mean values

new_df[['DVTC20X', 'PERWT20F']] = impute_outliers_IQR(new_df[['DVTC20X', 'PERWT20F']])

## lets check if the function is successful
new_df.describe()[['DVTC20X', 'PERWT20F']]    

## As we can see, there are still 21363 rows, the min is our lower limit and the max is the upper limit. 
## Now the mean for both variables when compared with the max values indicates that outliers have been removed. 
## That means the function was successful. 

,DVTC20X,PERWT20F
count,21363.000000,21363.000000
mean,217.601725,11512.988926
std,154.123429,5644.942249
min,0.000000,0.000000
25%,110.000000,7435.709198
50%,184.380000,11021.997937
75%,320.000000,14502.886948
max,635.000000,28313.120141


In [13]:
## Check redundancy of attributes by using correlations tests and remove redundant attributes.

# first we create a correlation matrix
# then we select the upper triangular matrix

matrix_cor = new_df.corr()
print(matrix_cor)

              DUPERSID   EVNTIDX  JUSTXRAY_M18   EVENTRN  ROOTCANL_M18  \
DUPERSID      1.000000  1.000000     -0.000743 -0.927693      0.001740   
EVNTIDX       1.000000  1.000000     -0.000743 -0.927693      0.001740   
JUSTXRAY_M18 -0.000743 -0.000743      1.000000 -0.002860      0.827131   
EVENTRN      -0.927693 -0.927693     -0.002860  1.000000     -0.006470   
ROOTCANL_M18  0.001740  0.001740      0.827131 -0.006470      1.000000   
DVDATEMM     -0.070670 -0.070670     -0.012919  0.382404     -0.010195   
DVDATEYR           NaN       NaN           NaN       NaN           NaN   
PANEL         0.997007  0.997007     -0.001446 -0.928496      0.000804   
DENTPROX      0.000744  0.000744      0.829723 -0.006437      0.967566   
GENDENT_M18  -0.012176 -0.012176      0.334799  0.006320      0.338517   
GENDENT_M18  -0.012176 -0.012176      0.334799  0.006320      0.338517   
DVTC20X      -0.005344 -0.005344     -0.031636  0.010469     -0.024162   
PERWT20F      0.206729  0.206729      

In [15]:
    ## upper triangular
    tri_upper = matrix_cor.where(np.triu(np.ones(matrix_cor.shape),k=1).astype(np.bool))
    print(tri_upper)

              DUPERSID  EVNTIDX  JUSTXRAY_M18   EVENTRN  ROOTCANL_M18  \
DUPERSID           NaN      1.0     -0.000743 -0.927693      0.001740   
EVNTIDX            NaN      NaN     -0.000743 -0.927693      0.001740   
JUSTXRAY_M18       NaN      NaN           NaN -0.002860      0.827131   
EVENTRN            NaN      NaN           NaN       NaN     -0.006470   
ROOTCANL_M18       NaN      NaN           NaN       NaN           NaN   
DVDATEMM           NaN      NaN           NaN       NaN           NaN   
DVDATEYR           NaN      NaN           NaN       NaN           NaN   
PANEL              NaN      NaN           NaN       NaN           NaN   
DENTPROX           NaN      NaN           NaN       NaN           NaN   
GENDENT_M18        NaN      NaN           NaN       NaN           NaN   
GENDENT_M18        NaN      NaN           NaN       NaN           NaN   
DVTC20X            NaN      NaN           NaN       NaN           NaN   
PERWT20F           NaN      NaN           NaN      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [17]:
## print the redundant columns to be removed
to_drop = [column for column in tri_upper.columns if any(tri_upper[column] > 0.95)]
print(); print(to_drop)


['EVNTIDX', 'PANEL', 'DENTPROX', 'GENDENT_M18', 'GENDENT_M18']


In [35]:
## drop the highly correrated columns

new_df_created = new_df.drop(new_df[['EVNTIDX', 'PANEL', 'DENTPROX', 'GENDENT_M18', 'GENDENT_M18']], axis=1)
new_df_created.head(n=1000)

,DUPERSID,JUSTXRAY_M18,EVENTRN,ROOTCANL_M18,DVDATEMM,DVDATEYR,DVTC20X,PERWT20F
0,2320012102,2,6,2,2,2020,285.000000,1960.941227
1,2320012102,-8,7,-8,8,2020,285.000000,1960.941227
2,2320012102,2,7,2,12,2020,236.920000,1960.941227
3,2320024102,2,6,2,1,2020,127.730000,4054.825730
4,2320024102,2,7,2,10,2020,127.730000,4054.825730
...,...,...,...,...,...,...,...,...
995,2321518102,1,6,2,3,2020,453.295531,4591.221938
996,2321518102,2,7,2,12,2020,225.000000,4591.221938
997,2321524101,2,6,2,3,2020,125.000000,3990.155101
998,2321532101,1,6,2,8,2020,100.000000,12907.763863


In [34]:
## Normalized data

new_df_scaled = preprocessing.normalize(new_df_created, axis=0)
new_df_scaled = pd.DataFrame(new_df_scaled)

new_df_scaled.head()

,0,1,2,3,4,5,6,7
0,0.006446,0.007322,0.009181,0.006629,0.001796,0.006842,0.007313,0.001046
1,0.006446,-0.029287,0.010711,-0.026516,0.007185,0.006842,0.007313,0.001046
2,0.006446,0.007322,0.010711,0.006629,0.010778,0.006842,0.006079,0.001046
3,0.006446,0.007322,0.009181,0.006629,0.000898,0.006842,0.003277,0.002164
4,0.006446,0.007322,0.010711,0.006629,0.008982,0.006842,0.003277,0.002164
